## 使用tf风格迁移

In [1]:
import matplotlib.pyplot as plt 
import tensorflow_hub as hub 
import tensorflow as tf
import numpy as np


In [ ]:
def load_image_local(image_path, image_size=(512, 512), preserve_aspect_ratio=True):
    """读取图片并转化为float32的numpy数组,并把像素值归一化到0-1之间
    """
    pass